In [ ]:
#Import earth engine
import ee

#Trigger the authentication flow
ee.Authenticate()

#Initialize the library
ee.Initialize(project='modis-410817')

In [ ]:
# Import geemap
import geemap as emap

In [ ]:
#Install ipygee
!pip install ipygee

In [ ]:
#Import ipygeepackage
from ipygee import *

In [ ]:
#Create a region of interest (ROI)
roi = ee.Geometry.Rectangle([10.97, 47.40, 11.47, 47.50]);

#Load MODIS Terra Snow Cover Daily Global 500m
imageCollection = ee.ImageCollection("MODIS/061/MOD10A1").select('NDSI_Snow_Cover');

#Filter the image collection for the specified date ranges and region of interest
filteredCollection = imageCollection \
  .filter(ee.Filter.calendarRange(2001, 2002, 'year')) \
  .filter(ee.Filter.Or(
    ee.Filter.calendarRange(1, 3, 'month'),
    ee.Filter.calendarRange(10, 12, 'month')
  )) \
  .filterBounds(roi)

In [ ]:
# Get the image IDs
image_ids = filteredCollection.aggregate_array('system:index').getInfo()
print('Total images', len(image_ids))

In [ ]:
#Visualization parameters
snow_cover_vis = {
    'min': 0.0,
    'max': 100.0,
    'palette': ['black', '0dffff', '0524ff', 'ffffff']
}

In [ ]:
# Get a list of images
image_list = filteredCollection.toList(filteredCollection.size())

#Clip for displaying in map
img = ee.Image(image_list.get(0)).clip(roi)

In [ ]:
#Create a center the map object
Map = emap.Map()
Map.center_object(roi, zoom=12)

# Add the layers to the map object.
Map.addLayer(img,snow_cover_vis,'Snow cover')

# Draw the rectangle on the map
Map.addLayer(roi, {'color': 'red'}, 'roi')

#Add a layer control panel to the map
Map.addLayerControl()

#Display the map
Map

In [ ]:
# Define a function to calculate the mean value for an image
def calculate_mean(image):
    mean = image.reduceRegion(ee.Reducer.mean(), roi).get('NDSI_Snow_Cover')
    return image.set('mean_value', mean)

# Map the function over the clipped image collection
clipped_collection_with_mean = filteredCollection.map(calculate_mean)

# Create a list of numeric IDs
id_list = ee.List.sequence(1, clipped_collection_with_mean.size())

# Convert the image collection to a feature collection with numeric IDs
def map_function(id):
    image = clipped_collection_with_mean.toList(clipped_collection_with_mean.size()).get(ee.Number(id).subtract(1))
    date = ee.Date(ee.Image(image).get('system:time_start'))
    return ee.Feature(None, {
        'ID_Numeric': id,
        'Date': date.format('YYYY-MM-dd'),
        'Mean_Value': ee.Image(image).get('mean_value')
    })

feature_collection = ee.FeatureCollection(id_list.map(map_function))

# Export the feature collection to a CSV file
task = ee.batch.Export.table.toDrive(
    collection=feature_collection,
    description='exported_data',
    folder='your_folder_name120',
    fileNamePrefix='exported_data',
    fileFormat='CSV'
)

# Start the export task
task.start()

In [ ]:
# Loop through the images
for i in range(image_list.length().getInfo()):
    img = ee.Image(image_list.get(i))

    # Cast all bands to Int16
    img = img.toUint16()

    # Define the export parameters
    export_params = {
        'image': img.select('NDSI_Snow_Cover'), #'image': img,
        'description': 'Image Export {}'.format(i + 1),
        'fileNamePrefix': img.id().getInfo(),
        'folder': 'Snow_Cover_M09_50',
        'scale': 500,
        'region': roi.getInfo()['coordinates'],
        'crs': 'EPSG:4326',
        'maxPixels': 1e10
    }

    # Start the export task
    task = ee.batch.Export.image.toDrive(**export_params)
    task.start()

    print('Started task: ', i + 1)

In [ ]:
##To check if any data has zero snow cover value
# Create a time series chart
ts_sum = chart.Image.series(**{
    'imageCollection': filteredCollection,
    'region':roi,
    'reducer':ee.Reducer.sum(),
    'scale':500,
   })

# Print the chart
ts_sum.renderWidget(width='1500px', height='500px')

In [ ]:
ts_sum.dataframe

In [ ]:
# Create a time series chart
ts_mean = chart.Image.series(**{
    'imageCollection': filteredCollection,
    'region':roi,
    'reducer':ee.Reducer.mean(),
    'scale':500
    })

# Print the chart
ts_mean.renderWidget(width='1500px', height='500px')

In [ ]:
ts_mean.dataframe